In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e1/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e1/test.csv')

In [ ]:
train.head()

In [ ]:
train.drop(['MedInc','AveOccup','Latitude','Longitude'], inplace=True, axis=1)
test.drop(['MedInc','AveOccup','Latitude','Longitude'], inplace=True, axis=1)

In [ ]:
x = train.drop("MedHouseVal",axis=1)
y = train['MedHouseVal']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(x_train,y_train)

In [ ]:
predictions= clf.predict(x_val)

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_val, predictions)
mse

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(x_train, y_train)

In [ ]:
predictions= gbr.predict(x_val)
gbrp = mean_squared_error(y_val, predictions)
gbrp

In [ ]:
search_space = {
    'n_estimators' : [100,200,300],
    'max_depth' : [3,6,9],
    'learning_rate' : [0.001,0.01,0.1,0.2,0.3],
}

In [ ]:
from sklearn.model_selection import GridSearchCV
GS = GridSearchCV(
    estimator = gbr,
    param_grid = search_space,
    scoring = ['r2', 'neg_root_mean_squared_error'],
    refit = 'r2',
    cv = 3,
    verbose = 4,
    n_jobs = -1,
)

In [ ]:
GS.fit(x_train,y_train)

In [ ]:
print(GS.best_params_)

In [ ]:
tunedpredictions = GS.predict(x_val)
tundederror = mean_squared_error(y_val, tunedpredictions)
tundederror

In [ ]:
submission_prediction = GS.predict(test)
idss = test["id"]

In [ ]:
ddf = pd.DataFrame({"id":idss.values,
                   "MedHouseVal":submission_prediction})

In [ ]:
ddf.to_csv("submissio.csv", index = False)